# Index compressed images
After image compression, put the images back into the index

## Requires
Currently requires more than 1024 MB worker memory especially with large DICOM files.
- Recommend a classic UC enabled cluster

In [0]:
%pip install --quiet numpy==1.26.4 pydicom==3.0.1 pylibjpeg>=2.0 pylibjpeg-openjpeg>=2.0

In [0]:
dbutils.library.restartPython()

### Config
This will create and save to a local config.yaml file

In [0]:
import yaml
cfg = yaml.safe_load(open("config.yaml", "r"))

# Output path for the compressed DICOM file
dbutils.widgets.text("output_path", cfg.get("output_path"))
output_path = dbutils.widgets.get("output_path")

dbutils.widgets.text("table", cfg.get("table"))
table = dbutils.widgets.get("table")

yaml.safe_dump({"output_path": output_path, "table": table}, open("config.yaml", "w"))

In [0]:
from dbx.pixels import Catalog
from dbx.pixels.dicom import DicomMetaExtractor # The Dicom transformers

catalog = Catalog(spark, table=table)
catalog_df = catalog.catalog(path=output_path)
meta_df = DicomMetaExtractor(catalog=catalog).transform(catalog_df)

In [0]:
meta_df.write.mode('overwrite').saveAsTable(table)

In [0]:
%sql select * from IDENTIFIER(:table)